# 데이터사이언스융합학과 2024710658 박창준

# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [ ]:
# pandas 불러오기
import pandas as pd
# csv 파일을 읽어와 df형식의 객체 생성
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# to_numpy() : 데이터형을 numpy 배열로 변경
# pd 형식의 데이터를 numpy로 변환
# 알콜, 당도, ph를 data 객체 numpy 형식으로 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# class 컬럼을 target 객체에 저장
target = wine['class'].to_numpy()

In [ ]:
# train_test_split 불러오기
from sklearn.model_selection import train_test_split

# train_test_split 함수 이용해서 train x, test x, train y, test y 데이터로 분리
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
# train_test_split 함수를 다시 이용해서 train x, val x, train y, val y 데이터로 분리
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
# 학습 데이터와 검증 데이터의 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
# DecisionTreeClassifier 불러오기
from sklearn.tree import DecisionTreeClassifier
# DecisionTreeClassifier 객체 생성
dt = DecisionTreeClassifier(random_state=42)
# DecisionTreeClassifier 학습
dt.fit(sub_input, sub_target)

# 학습 점수 출력
print(dt.score(sub_input, sub_target))
# 테스트 점수 출력
print(dt.score(val_input, val_target))
# 학습 점수가 너무 높음 -> 오버피팅 발생

0.9971133028626413
0.864423076923077


## 교차 검증

In [ ]:
# cross_validate 불러오기
# 교차검증을 수행할때 활용
from sklearn.model_selection import cross_validate

# cross_validate 객체 생성
# cross_validate(모델객체, train x, train y)
scores = cross_validate(dt, train_input, train_target)
# 교차검증 결과 출력
# fit_time : 학습하는 시간을 의미
# score_time : 검증하는 시간을 의미
# test_score : 검증 폴드의 점수
print(scores)
# 교차검증의 최종 점수는 test_score의 평균값

{'fit_time': array([0.01430035, 0.01063991, 0.01435041, 0.02206588, 0.02361274]), 'score_time': array([0.00195861, 0.00140858, 0.00308847, 0.00306129, 0.00713658]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# numpy 불러오기
import numpy as np
# 교차검증 최종 점수 출력
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# StratifiedKFold 불러오기
# cross_validate() 함수는 학습 모델이 분류 모델인 경우 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold

# cross_validate 객체 생성
# cross_validate(모델객체, train x, train y)
# cv=StratifiedKFold() : dt가 분류모델이라 추가됨
# 레이블 비율에 맞게 교차검증 수행하기 위함
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

# 교차검증 최종 점수 출력
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 학습 데이터를 섞은 후 10-fold 교차검증 수행
# shuffle=True : 데이터 섞기
# n_splits=10 : fold 수 지정
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# cross_validate 객체 생성
# cv=splitter 파라미터로 추가됨
scores = cross_validate(dt, train_input, train_target, cv=splitter)

# 교차검증 최종 점수 출력
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [ ]:
# GridSearchCV 불러오기
# GridSearchCV : 여러 파라미터 조합 실험을 수행해줌
from sklearn.model_selection import GridSearchCV

# dt의 파라미터 중 하나인 min_impurity_decrease에 대한 실험
# 최적값을 알기 위해 list를 value로 하는 dict 생성
# 'min_impurity_decrease' : 노드를 분할하기 위한 불순도 감소 최소량
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# GridSearchCV 객체 생성
# 실험 모델(DecisionTreeClassifier)과 실험할 변수(params)를 전달
# n_jobs=-1 : 모든 CPU 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
# GridSearchCV 학습
# 학습을 수행하면 params 값을 순차적으로 학습
# fold 수가 default 값으로 5로 지정되어 있음
# params 5번 x 5번의 fold = 25번의 학습 수행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# best_estimator_ : 가장 높은 검증점수를 받은 모델이 저장되어 있음
# 검증 점수가 가장 높은 점수를 dt에 저장
dt = gs.best_estimator_
# dt 학습 점수 출력
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
# best_params_ : 가장 최적의 매개변수가 저장되어 있음
# gs로 찾은 최적의 매개변수 출력
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
# cv_results_['mean_test_score']: 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장되어 있음
# 5번 fold의 교차검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# np.argmax() 통해 교차검증 점수 중 가장 큰값의 index를 저장
best_index = np.argmax(gs.cv_results_['mean_test_score'])
# 해당 값을 출력하여 best_params_와 일치하는지 확인
print(gs.cv_results_['params'][best_index])
# 일치함

{'min_impurity_decrease': 0.0001}


In [ ]:
# params에 더 많은 파라미터 부여
# 'min_impurity_decrease' : 노드를 분할하기 위한 불순도 감소 최소량, 9가지
# 'max_depth' : 트리의 깊이 제한, 15가지
# 'min_samples_split' : 노드를 나누기 위한 최소 샘플의 수, 10가지
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
# default 5 교차검증 수행
# 9 x 1 5x 10 x 5 = 6750개의 모델 조합 수행

In [ ]:
# GridSearchCV 객체 생성
# 실험 모델(DecisionTreeClassifier)과 실험할 변수(params)를 전달
# n_jobs=-1 : 모든 CPU 코어 사용
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# GridSearchCV 학습
# 학습을 수행하면 params 값을 순차적으로 학습
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# best_params_ : 가장 최적의 매개변수가 저장되어 있음
# gs로 찾은 최적의 매개변수 출력
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
# cv_results_['mean_test_score']: 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장되어 있음
# 5번 fold의 교차검증으로 얻은 점수 중 최댓값 출력
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [ ]:
# uniform, randint 불러오기
from scipy.stats import uniform, randint

In [ ]:
# 0~10 사이의 범위를 갖는 randint객체 생성
rgen = randint(0, 10)
# rgen에서 10개의 숫자를 샘플링
rgen.rvs(10)

array([3, 6, 7, 2, 1, 2, 4, 0, 0, 0])

In [ ]:
# rgen에서 1000개의 숫자를 샘플링
# return_counts=True : 샘플링한 숫자의 갯수까지 출력
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104, 128,  89,  83,  94,  82, 105, 120, 104,  91]))

In [ ]:
# 0 ~ 1 사이의 범위를 갖는 uniform객체 생성
ugen = uniform(0, 1)
# ugen에서 10개의 숫자를 샘플링
ugen.rvs(10)

array([0.57050037, 0.04572821, 0.51557215, 0.15851336, 0.88912426,
       0.52207822, 0.22405458, 0.82022537, 0.82153998, 0.41788426])

In [ ]:
# params에 더 많은 파라미터 부여
# 'min_impurity_decrease' : 노드를 분할하기 위한 불순도 감소 최소량
# 'max_depth' : 트리의 깊이 제한
# 'min_samples_split' : 노드를 나누기 위한 최소 샘플의 수
# 'min_samples_leaf' : 리프 노드가 되기 위한 최소 샘플의 수
# 해당 파라미터에 임의의 범위값을 부여
# 시스템 자원이 허락하는 한 범위를 최대한 크게 설장하는게 좋음
# 그래야 최적의 조합을 찾을 수 있음
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
# RandomizedSearchCV 불러오기
from sklearn.model_selection import RandomizedSearchCV

# RandomizedSearchCV 객체 생성
# params를 받아옴
# n_iter=100 : 훈련횟수 100회
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bd4a7df7970>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7bd49f973490>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bd49f9738b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bd49f973550>},
                   random_state=42)

In [ ]:
# 최적의 매개변수 조합 출력
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
# 최고의 교차검증 점수 출력
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
# 가장 최적인 모델을 저장
dt = gs.best_estimator_
# 최적인 모델의 학습 점수 출력
print(dt.score(test_input, test_target))
# 테스트 점수는 검증 점수보다 조금 작은것이 일반적임

0.86
